<a href="https://colab.research.google.com/github/mehdiiiii786/finance/blob/main/E_D_I_T_H_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPortfolioOpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 17.6 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pypfopt import expected_returns, risk_models, HRPOpt

In [ ]:
sector_map = {
# 💻 TECHNOLOGY (Growth)
    'NVDA': 'Technology', 'MSFT': 'Technology', 'AAPL': 'Technology', 'AMD': 'Technology',
    'AVGO': 'Technology', 'CRM': 'Technology', 'ORCL': 'Technology', 'ADBE': 'Technology',
    'INTC': 'Technology', 'QCOM': 'Technology', 'CSCO': 'Technology', 'PLTR': 'Technology',

    # 📡 COMM SERVICES (Internet/Media)
    'GOOGL': 'Comm Services', 'META': 'Comm Services', 'NFLX': 'Comm Services',
    'DIS': 'Comm Services', 'TMUS': 'Comm Services', 'VZ': 'Comm Services',

    # 🛍️ CONSUMER DISCRETIONARY (Spending)
    'AMZN': 'Cons. Disc.', 'TSLA': 'Cons. Disc.', 'HD': 'Cons. Disc.', 'MCD': 'Cons. Disc.',
    'NKE': 'Cons. Disc.', 'SBUX': 'Cons. Disc.', 'BKNG': 'Cons. Disc.', 'SHOP': 'Cons. Disc.',

    # 🛒 CONSUMER STAPLES (Safety)
    'WMT': 'Cons. Staples', 'COST': 'Cons. Staples', 'PG': 'Cons. Staples', 'KO': 'Cons. Staples',
    'PEP': 'Cons. Staples', 'TGT': 'Cons. Staples', 'PM': 'Cons. Staples',

    # ⚡ ENERGY (Oil/Gas)
    'XOM': 'Energy', 'CVX': 'Energy', 'COP': 'Energy', 'SLB': 'Energy',
    'EOG': 'Energy', 'OXY': 'Energy', 'MPC': 'Energy', 'VLO': 'Energy',

    # 🏦 FINANCIALS (Banks)
    'JPM': 'Financials', 'BAC': 'Financials', 'V': 'Financials', 'MA': 'Financials',
    'GS': 'Financials', 'MS': 'Financials', 'BLK': 'Financials', 'COIN': 'Financials',

    # 💊 HEALTHCARE (Pharma)
    'LLY': 'Healthcare', 'UNH': 'Healthcare', 'JNJ': 'Healthcare', 'MRK': 'Healthcare',
    'ABBV': 'Healthcare', 'PFE': 'Healthcare', 'VRTX': 'Healthcare', 'ISRG': 'Healthcare',

    # 🏭 INDUSTRIALS (Defense/Planes)
    'GE': 'Industrials', 'CAT': 'Industrials', 'BA': 'Industrials', 'UNP': 'Industrials',
    'HON': 'Industrials', 'RTX': 'Industrials', 'LMT': 'Industrials', 'DE': 'Industrials',

    # ⛏️ MATERIALS (Gold/Lithium)
    'LIN': 'Materials', 'FCX': 'Materials', 'NEM': 'Materials', 'SCCO': 'Materials'
}

In [ ]:
# 1. THE DATA :
tick = list(sector_map.keys())

In [ ]:
#2. Data ingestion
data = yf.download(tick, period = "1y", interval = "1d",  progress = False)

/tmp/ipython-input-388476774.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tick, period = "1y", interval = "1d",  progress = False)


In [ ]:
if isinstance(data.columns, pd.MultiIndex):  #ma
  data =  data.stack(level=1).reset_index(level =1).pivot(columns = 'Ticker')

/tmp/ipython-input-1975848975.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data =  data.stack(level=1).reset_index(level =1).pivot(columns = 'Ticker')


In [ ]:
close_prices = data['Close']
volume_data = data['Volume']

In [ ]:
close_prices

Ticker,AAPL,ABBV,ADBE,AMD,AMZN,AVGO,BA,BAC,BKNG,BLK,...,TMUS,TSLA,UNH,UNP,V,VLO,VRTX,VZ,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-29,236.261444,176.749802,515.929993,137.179993,207.889999,160.505783,155.440002,46.417641,5164.931641,1002.170593,...,243.146500,345.160004,597.247314,238.976898,312.857880,134.833435,468.130005,41.483875,91.606781,113.783966
2024-12-02,238.511276,175.629028,516.200012,142.059998,210.710007,164.892761,156.539993,45.958447,5187.072754,999.534851,...,241.059052,357.089996,595.602966,235.187027,314.416809,135.861069,465.730011,41.025440,91.745438,113.677872
2024-12-03,241.557480,175.368134,516.260010,141.979996,213.440002,166.516846,155.000000,45.743504,5177.243652,1018.043884,...,241.088593,351.420013,592.382751,231.485062,310.802490,133.272583,456.970001,41.006729,92.607040,113.504234
2024-12-04,241.915863,170.498398,536.489990,143.990005,218.160004,168.903442,158.279999,45.303852,5205.818848,1014.124573,...,239.345795,357.929993,597.824768,229.541275,307.714417,129.976395,463.980011,39.781113,93.537956,110.234253
2024-12-05,241.945740,170.218185,538.219971,141.360001,220.550003,168.814301,156.669998,45.919369,5250.279297,1028.776733,...,240.192581,369.489990,566.680115,229.179886,306.900177,128.987564,459.920013,39.809174,94.379753,110.716553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-21,271.489990,236.279999,324.190002,203.779999,220.690002,340.200012,179.699997,51.560001,4768.000000,1014.719971,...,208.447281,391.089996,319.970001,226.220001,327.980011,173.449997,426.760010,41.230000,105.320000,117.080002
2025-11-24,275.920013,229.509995,318.730011,215.050003,226.279999,377.959991,179.119995,51.930000,4875.370117,1019.489990,...,206.317825,417.779999,319.049988,224.500000,329.299988,174.369995,424.000000,40.209999,104.059998,115.970001
2025-11-25,276.970001,231.800003,319.549988,206.130005,229.669998,385.029999,182.440002,52.480000,4906.240234,1029.770020,...,205.880005,419.399994,326.279999,229.130005,334.529999,174.100006,429.820007,40.590000,107.000000,114.510002


In [ ]:
#3. les signals (RSI, sma,, Zscore, volume)
analysis =  pd.DataFrame(index = tick)
current_prices = close_prices.iloc[-1]
current_volume = volume_data.iloc[-1]

In [ ]:
#simple moving average :
sma_200 = close_prices.rolling(window = 200).mean().iloc[-1]
analysis['Trend'] = np.where (current_prices > sma_200, "BULL", "BEAR")

In [ ]:
#z-score :
sma_20 = close_prices.rolling(window = 20).mean().iloc[-1]
std_20 = close_prices.rolling(window = 20).std().iloc[-1]
analysis['Z-score']= (current_prices - sma_20) / std_20

In [ ]:
#RSI :
delta = close_prices.diff()
gain =(delta.where(delta > 0,0)).fillna(0)
loss =(-delta.where(delta < 0,0)).fillna(0)
avg_gain =gain.rolling(window = 14).mean().iloc[-1]
avg_loss= loss.rolling(window =14).mean().iloc[-1]

In [ ]:
rs = avg_gain / avg_loss
analysis['RSI']= 100 - (100/(1+rs))

In [ ]:
# volume force
vol_sma_20 = volume_data.rolling(window = 20).mean().iloc[-1]
analysis['Volume_Force'] = current_volume / vol_sma_20

In [ ]:
#4. Strategies :
analysis['Signal'] = 'IGNORE'
#A- l3orooooood :

analysis.loc[ (analysis['Trend'] == 'BULL') & (analysis['Z-score']< -1.5)& (analysis ['RSI']< 45) & (analysis['Volume_Force']> 0.8), 'Signal'] = 'BUY'

In [ ]:
#B- ROKOB L2AMWAAAJ :

analysis.loc[(analysis['Trend'] == 'BULL') & (analysis ['Z-score'] > 0.5) & (analysis ['Z-score'] < 2.5) & (analysis['RSI'] > 50) & (analysis['RSI'] < 75) & (analysis['Volume_Force']> 1.2), 'Signal'] =  "MOMENTUM"

In [ ]:
candidates =  analysis[analysis['Signal'] != "IGNORE"].index.tolist()

In [ ]:
candidates

[]

In [ ]:
analysis

,Trend,Z-score,RSI,Volume_Force,Signal
NVDA,BULL,-1.358736,41.205240,0.559144,IGNORE
MSFT,BULL,-0.360604,47.743171,0.412628,IGNORE
AAPL,BEAR,1.968447,67.228000,0.395434,IGNORE
AMD,BULL,-0.894451,43.594466,0.342078,IGNORE
AVGO,BULL,2.353192,70.650410,0.590948,IGNORE
...,...,...,...,...,...
DE,BULL,-1.191417,48.023940,1.064019,IGNORE
LIN,BEAR,-0.922808,37.527511,0.303909,IGNORE
FCX,BEAR,2.168368,63.175389,0.594211,IGNORE
NEM,BULL,1.278928,61.849778,0.508194,IGNORE


In [ ]:
#5- the optimizer : (hierarchical risk parity) :
if len(candidates) > 0 :
  print('running the opimization engine...')
  subset_prices =close_prices[candidates]
  S= risk_models.sample_cov(subset_prices)

  hrp = HRPOpt(returns= None, cov_matrix=S)
  hrp_weights = hrp.optimize()
  clean_weights = hrp.clean_weights()
  print("\n🏆 ULTIMATE PORTFOLIO ALLOCATION (Cyber Monday):")

  weights_df = pd.Seies(clean_weights).sort_values(ascending = False)
  for ticker, weight in weights_df.item():
    if weight > 0 :
      print(f"👉 {ticker}: {weight*100:.2f}%  (${(weight*100000):.0f})")
else:
    print("\n⚠️ NO SIGNALS DETECTED.")
    print("The model recommends: 100% CASH. Do not force trades.")





⚠️ NO SIGNALS DETECTED.
The model recommends: 100% CASH. Do not force trades.
